# <p style="text-align: center;"> MMOCR Installation Check for TextyBeast</p>
_________________________________________________________________________________________________________________

This notebook is based on the official colab tutorial for using MMOCR. However, this adaptation is intended to simply serve as a means to check that your mmocr installation worked when installing the TextyBeast package...

To check your installation, please run each of the cells below. If these run without error, your mmocr environment is functioning.

#### NOTE: MAKE SURE THAT YOU ACTIVATED THE MMOCR PYTHON VIRTUAL ENVIRONMENT PRIOR TO OPENING/RUNNING THIS NOTEBOOK!
__________________________________________________________________________________________________________________

If you have not done so, please do this by first navigating to the TextyBeast main directory.

Then run: 
- cd envs/ocr/
- source env/bin/activate
________________________________________________________________________________________________________

#### _________________________________________________________________________________________________________________

## 1) Check PyTorch version
The call in the cell below should output information regarding the pytorch installation in the mmocr virtual python environment installed for TextyBeast. The output should look something like this:

==================================================================================================================
Name: torch

Version: 1.13.1+cu117

Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration

Home-page: https://pytorch.org/

Author: PyTorch Team

Author-email: packages@pytorch.org

License: BSD-3

Location: /mnt/76cfc2bb-b830-47a1-9a9c-0d28b8a1efab/python_projects/installTesting/dataScraping/envs/ocr/env/lib/python3.8/site-packages

Requires: typing-extensions

Required-by: torchaudio, torchvision

==================================================================================================================

If this runs without error, torch is installed and the version info is here for reference should you need it. You can then proceed to the next step.

In [ ]:
!pip show torch


#### _________________________________________________________________________________________________________________
## 2) Check Installed Dependencies Versions

The following calls check whether a number of dependencies necessary for MMOCR to function installed correctly. Namely it checks whether:
1) CUDA is installed/available (this is necessary for running gpu versions of mmocr models on NVIDIA cards)
2) mmdet is installed
3) mmcv is installed
4) whether mmocr is installed

Your output should look something like this:

==================================================================================================================
1.13.1+cu117 True

2.28.2

1.7.1

11.7

GCC 9.3

0.6.3

==================================================================================================================

If this runs without error, you have the dependencies installed and can then proceed to the next step ...


In [ ]:
# 1) Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# 2) Check MMDetection installation
import mmdet
print(mmdet.__version__)

# 3) Check mmcv installation
import mmcv
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(mmcv.__version__)
print(get_compiling_cuda_version())
print(get_compiler_version())

# 4) Check mmocr installation
import mmocr
print(mmocr.__version__)

#### _________________________________________________________________________________________________________________

## 3) Test out some MMOCR calls:

Below are some test calls to mmocr from their demo notebook for running inference and recognition on a number of demo images. If you are able to run the cells below without error, your mmocr installation is functioning...

#### _________________________________________________________________________________________________________________
### 3a: Inference

We provide an easy-to-use inference script, `mmocr/utils/ocr.py`, that can be either called through command line or imported as an object (the `MMOCR` class inside). In this notebook, we choose the latter option for ease of demonstration. You can check out its full usage and examples in our [official documentation](https://mmocr.readthedocs.io/en/latest/demo.html).

In [ ]:
# First, lets set up some path variables..

####################################################################################
# Note: This must be run prior to later cells which require these path locations...#
####################################################################################

import os

# get the full path to the demo folder in the mmocr directory...
demoFileLoc = os.getcwd()
demoFileLoc = demoFileLoc+"/TextyBeast_MMOCR_Install_Check.ipynb"

# go up a level and get the full path to the mmocr install directory...
%cd ..
installDir=os.getcwd()

print("")
print("")
print("PATH INFO:")
print("==========================================================================================================")
print("This demo is saved at:")
print(demoFileLoc)
print("")
print("The full path to the mmocr installation in this environment is:")
print(installDir)
print("==========================================================================================================")

#### _________________________________________________________________________________________________________________
### 3b: Perform Inference with a Pretrained Text Recognizer 

We now demonstrate how to inference on a [demo text recognition image](https://github.com/open-mmlab/mmocr/raw/main/demo/demo_text_recog.jpg) with a pretrained text recognizer using command line. SAR text recognizer is used for this demo, whose checkpoint can be found in the [official documentation](https://mmocr.readthedocs.io/en/latest/textrecog_models.html#show-attend-and-read-a-simple-and-strong-baseline-for-irregular-text-recognition). But you don't need to download it manually -- Our inference script handles these cumbersome setup steps for you! 

Run the following command and the recognition result will be saved to `recogDemoOut/out_demo_text_recog.jpg`. We will visualize the result in the end.

In [ ]:
from mmocr.utils.ocr import MMOCR
mmocr = MMOCR(det=None, recog='SAR')
recogDemoTxtPath=installDir+"/demo/demo_text_recog.jpg"
outputDir=installDir+"/recogDemoOut"
mmocr.readtext(recogDemoTxtPath, print_result=True, output=outputDir)

In [ ]:
# Visualize the results
import matplotlib.pyplot as plt
import mmcv

outFile=installDir+"/recogDemoOut/out_demo_text_recog.png";
predicted_img = mmcv.imread(outFile)
plt.imshow(mmcv.bgr2rgb(predicted_img))
plt.show()

#### _________________________________________________________________________________________________________________
### 3c: Perform Inference with a Pretrained Text Detector 

Next, we perform inference with a pretrained TextSnake text detector and visualize the bounding box results for the demo text detection image provided in [demo_text_det.jpg](https://raw.githubusercontent.com/open-mmlab/mmocr/main/demo/demo_text_det.jpg).

In [ ]:
from mmocr.utils.ocr import MMOCR
mmocr = MMOCR(det='TextSnake', recog=None)
txtDetectImgFile=installDir+"/demo/demo_text_det.jpg"
outDir='tsnakeOut/'
_ = mmocr.readtext(txtDetectImgFile, output=outDir)

In [ ]:
# Visualize the results
import matplotlib.pyplot as plt
outFile=outDir+"/out_demo_text_det.png"
predicted_img = mmcv.imread(outFile)
plt.figure(figsize=(9, 16))
plt.imshow(mmcv.bgr2rgb(predicted_img))
plt.show()

#### _________________________________________________________________________________________________________________
### 3d: Perform end-to-end OCR with pretrained recognizer and detector

With the help of `ocr.py`, we can easily combine any text detector and recognizer into a pipeline that forms a standard OCR step. Now we build our own OCR pipeline with TextSnake and SAR and apply it to [demo_text_ocr.jpg](https://raw.githubusercontent.com/open-mmlab/mmocr/main/demo/demo_text_ocr.jpg).

In [ ]:
from mmocr.utils.ocr import MMOCR
mmocr = MMOCR(det='TextSnake', recog='SAR')
outDir="End2EndOutputs"
inFile=installDir+"/demo/demo_text_ocr.jpg"
mmocr.readtext(inFile, print_result=True, output='End2EndOutputs')

In [ ]:
# Visualize the results
import matplotlib.pyplot as plt
predicted_img = mmcv.imread('./End2EndOutputs/out_demo_text_ocr.png')
plt.figure(figsize=(9, 16))
plt.imshow(mmcv.bgr2rgb(predicted_img))
plt.show()

#### _________________________________________________________________________________________________________________

#### If were able to run all of the tests and examples above, your mmocr install is working ...